In [4]:
import pandas as pd
from datetime import datetime
import re
import datetime

In [5]:
def mark_is_smoke(data, df):
    """
    data就是如上面的例子一样的一段文本
    df就是你要打标签的数据集
    返回值是打好标签的数据集
    """
    data = data.replace(' ', '') # 去掉空格
    match = re.search(r"(\d+)月(\d+)日(\d+):(\d+)-(\d+):(\d+)", data)  # 正则匹配
    month, day, start_hour, start_minute, end_hour, end_minute = map(int, match.groups())  # 转换成int

    current_year = datetime.datetime.now().year # 获取当前年份
    start_time = datetime.datetime(current_year, month, day, start_hour, start_minute)  # 构造开始时间
    end_time = datetime.datetime(current_year, month, day, end_hour, end_minute)      # 构造结束时间

    formatted_start_time = start_time.strftime("%Y/%#m/%#d %H:%M")  # Format start time as "2023/5/2 16:09"
    formatted_end_time = end_time.strftime("%Y/%#m/%#d %H:%M")      # Format end time as "2023/5/2 16:09"

    # Create 'is_smoke' column if it doesn't exist
    if 'is_smoke' not in df.columns:
        df['is_smoke'] = 0
        
    df['is_smoke'] = df.apply(lambda row: 1 if (formatted_start_time <= row['Timestamp'] <= formatted_end_time or row['is_smoke'] == 1) else 0, axis=1)

    return df

In [6]:
# 读取CSV文件
df = pd.read_csv('alldata2.csv')

# 数据列表
data_list = [
    "5月2日 9:31-9:52 第一次吸烟（纸烟）",
    "5月2日 13:10-13:25 第二次吸烟（纸烟+电子烟）",
    "5月2日 16:54-17:13 第三次吸烟（电子烟）",
    "5月3日 9:01-9:17 第一次吸烟（纸烟）",
    "5月3日 10:03-10:36 第二次吸烟（电子烟）",
    "5月3日 12:35-12:43 第三次吸烟（纸烟）",
    "5月4日 10:08-10:17 第一次抽烟（纸烟）",
    "5月4日 11:16-11:34 第二次抽烟（电子烟）",
    "5月4日 12:03-12:20 第三次抽烟（电子烟）",
    "5月4日 14:10-14:17 第四次抽烟（纸烟）",
    "5月4日 17:31-17:36 第五次抽烟（纸烟）",
    "5月7日 15:48-15:53 第一次抽烟（纸烟）",
    "5月9日 11:56-12:09 第一次抽烟（纸烟）",
    "5月9日 17:45-17:52 第二次抽烟（纸烟）",
    "5月10日 16:58-17:07 第一次抽烟（纸烟）"
]

# 循环处理数据
for data in data_list:
    mark_is_smoke(data, df)

print(df)

# 保存回CSV文件
df.to_csv('modified_data.csv', index=False)

             Timestamp  Accelaration-X  Accelaration-Y  Accelaration-Z  \
0       2023/5/2 15:43       -3.949597        7.128716       -5.204647   
1       2023/5/2 15:43       -3.649501        7.111816       -5.736820   
2       2023/5/2 15:43       -3.680014        6.884367       -5.928705   
3       2023/5/2 15:43       -3.962709        6.756036       -5.858173   
4       2023/5/2 15:43       -3.694737        7.466803       -5.224735   
...                ...             ...             ...             ...   
645920  2023/5/15 6:50       -0.014176        5.888215        7.646384   
645921  2023/5/15 6:50       -1.133114        4.530988        8.576385   
645922  2023/5/15 6:50       -0.611299        4.576673        8.682230   
645923  2023/5/15 6:50       -0.355301        4.325300        8.772133   
645924  2023/5/15 6:50       -0.625549        4.351750        8.739501   

        AngularVelocity-X  AngularVelocity-Y  AngularVelocity-Z  latitude  \
0                0.196320         

In [11]:
### 分割出吸烟和不吸烟的数据片段

def split_data_into_segments(df):
    segment_indices = []
    current_segment = None
    for index, row in df.iterrows():
        if current_segment is None:
            if row['is_smoke'] == 1:
                current_segment = []
                current_segment.append(index)
        else:
            if row['is_smoke'] == 0:
                current_segment.append(index)
                segment_indices.append(current_segment)
                current_segment = None
            else:
                current_segment.append(index)
    if current_segment is not None:
        segment_indices.append(current_segment)

    # Save each segment as a separate CSV file
    for i, segment in enumerate(segment_indices):
        segment_df = df.loc[segment]
        segment_df.to_csv(f"segment_{i+1}.csv", index=False)
    return segment_df

split_data_into_segments(df)